In [7]:
import pandas as pd
from IPython.display import display, HTML


import base
import fund_scanner.common_tools.database as db
import fund_scanner.business.grab_data as grab_data
engine = db.get_sqlalchemy_engine()

In [18]:
url = 'http://fund.eastmoney.com/manager/%s.html'

with db.get_connection() as cursor:
    sql = 'select * from managers limit 0,50'
    cursor.execute(sql)
    managers = cursor.fetchall()
    print(managers[0])
    
    for manager in managers:
        
        

        print(manager['managers_code'])

        df = pd.read_html(url%manager['managers_code'])[1]

        #len(df)

        if df.iloc[0,-1][-1] != '%' and df.iloc[0,-1][-1] != '-':
            print('抓取到的格式错误！')
            display(df)

        df['start_date'] = df.loc[:,'任职时间'].apply(lambda x: x.split(' ~ ')[0])
        df['end_date'] = df.loc[:,'任职时间'].apply(lambda x: x.split(' ~ ')[1].replace('至今', '9999-9-9'))
        df['funds_code'] = df.loc[:,'基金代码'].apply(lambda x: str(x).zfill(6))

        codes = df['基金代码'].tolist()
        codes = ','.join([str(w) for w in codes])

        df_funds = pd.read_sql('select * from funds where funds_code in (%s)'%(codes), engine)

        df = pd.merge(df, df_funds, on='funds_code', how='left')

        funds_need_update = df[df['funds_type'].isnull()]['funds_code'].tolist()
        if len(funds_need_update)>1:
            grab_data.update_funds(funds_need_update)

        df_funds = pd.read_sql('select * from funds where funds_code in (%s)'%(codes), engine)

        df = pd.merge(df, df_funds, on='funds_code', how='left')

        display(df)

        for row in df.iterrows():
            start_date = row[1]['start_date']
            end_date = row[1]['end_date']
            #funds_code = row[1][]
            break



{'managers_gendar_guess': None, 'managers_photo': None, 'managers_id': 1, 'managers_name': '艾小军', 'managers_code': '30277862', 'working_since_date': datetime.date(2014, 1, 13)}
30277862


,基金代码,基金名称,相关链接,基金类型,规模（亿元）,任职时间,任职天数,任职回报,start_date,end_date,...,funds_type_x,funds_start_date_x,update_time_x,historical_price_time_x,funds_id_y,funds_name_full_y,funds_type_y,funds_start_date_y,update_time_y,historical_price_time_y
0,1789,国泰量化收益灵活配置混合,估值图基金吧档案,混合型,2.13,2017-05-19 ~ 至今,73天,7.50%,2017-05-19,9999-9-9,...,混合型,2017-05-19,2017-07-31 04:11:03,2017-07-31 15:26:07,4182,国泰量化收益灵活配置混合,混合型,2017-05-19,2017-07-31 04:11:03,2017-07-31 15:26:07
1,4253,国泰黄金ETF联接C,估值图基金吧档案,联接基金,0.00,2017-05-02 ~ 至今,90天,-2.05%,2017-05-02,9999-9-9,...,联接基金,2017-05-02,2017-07-29 10:49:11,2017-07-31 19:55:14,857,国泰黄金ETF联接C,联接基金,2017-05-02,2017-07-29 10:49:11,2017-07-31 19:55:14
2,501016,国泰中证申万证券行业指数,估值图基金吧档案,股票指数,0.10,2017-04-27 ~ 至今,95天,7.25%,2017-04-27,9999-9-9,...,股票指数,2017-04-27,2017-07-31 12:54:02,2017-07-31 16:19:10,4704,国泰中证申万证券行业指数,股票指数,2017-04-27,2017-07-31 12:54:02,2017-07-31 16:19:10
3,501019,国泰国证航天军工指数,估值图基金吧档案,股票指数,1.68,2017-03-29 ~ 至今,124天,-15.80%,2017-03-29,9999-9-9,...,股票指数,2017-03-29,2017-07-31 12:34:02,2017-07-31 16:17:10,4684,国泰国证航天军工指数,股票指数,2017-03-29,2017-07-31 12:34:02,2017-07-31 16:17:10
4,199,国泰策略收益灵活配置混合,估值图基金吧档案,混合型,0.54,2017-03-06 ~ 至今,147天,1.77%,2017-03-06,9999-9-9,...,混合型,2013-08-12,2017-07-29 10:52:11,2017-07-31 18:05:10,896,国泰策略收益灵活配置混合,混合型,2013-08-12,2017-07-29 10:52:11,2017-07-31 18:05:10
5,20022,国泰策略价值灵活配置混合,估值图基金吧档案,混合型,1.77,2017-03-06 ~ 至今,147天,4.18%,2017-03-06,9999-9-9,...,混合型,2011-04-19,2017-07-31 01:32:03,2017-07-31 15:10:10,4023,国泰策略价值灵活配置混合,混合型,2011-04-19,2017-07-31 01:32:03,2017-07-31 15:10:10
6,512880,国泰中证全指证券公司ETF,估值图基金吧档案,ETF-场内,2.70,2016-07-26 ~ 至今,1年又5天,4.81%,2016-07-26,9999-9-9,...,ETF-场内,2016-07-26,2017-07-31 15:25:35,2017-07-31 13:44:04,99710,国泰中证全指证券公司ETF,ETF-场内,2016-07-26,2017-07-31 15:25:35,2017-07-31 13:44:04
7,512660,国泰中证军工ETF,估值图基金吧档案,ETF-场内,3.51,2016-07-26 ~ 至今,1年又5天,-12.90%,2016-07-26,9999-9-9,...,ETF-场内,2016-07-26,2017-07-31 20:31:27,2017-07-31 20:32:04,99997,国泰中证军工ETF,ETF-场内,2016-07-26,2017-07-31 20:31:27,2017-07-31 20:32:04
8,218,国泰黄金ETF联接A,估值图基金吧档案,联接基金,0.92,2016-04-13 ~ 至今,1年又109天,4.72%,2016-04-13,9999-9-9,...,联接基金,2016-04-13,2017-07-29 10:48:27,2017-07-31 19:55:07,850,国泰黄金ETF联接A,联接基金,2016-04-13,2017-07-29 10:48:27,2017-07-31 19:55:07
9,20011,国泰沪深300,估值图基金吧档案,股票指数,20.02,2015-04-10 ~ 至今,2年又113天,-7.48%,2015-04-10,9999-9-9,...,股票指数,2007-11-11,2017-07-31 06:43:03,2017-07-31 15:41:16,4334,国泰沪深300,股票指数,2007-11-11,2017-07-31 06:43:03,2017-07-31 15:41:16


30158947


,基金代码,基金名称,相关链接,基金类型,规模（亿元）,任职时间,任职天数,任职回报,start_date,end_date,...,funds_type_x,funds_start_date_x,update_time_x,historical_price_time_x,funds_id_y,funds_name_full_y,funds_type_y,funds_start_date_y,update_time_y,historical_price_time_y
0,823,银华高端制造业,估值图基金吧档案,混合型,3.40,2016-11-07 ~ 至今,266天,2.76%,2016-11-07,9999-9-9,...,混合型,2014-11-13,2017-07-29 09:49:10,2017-07-31 17:47:13,312,银华高端制造业,混合型,2014-11-13,2017-07-29 09:49:10,2017-07-31 17:47:13
1,1163,银华中国梦30股票,估值图基金吧档案,股票型,15.65,2015-04-29 ~ 至今,2年又94天,-2.90%,2015-04-29,9999-9-9,...,股票型,2015-04-29,2017-07-30 10:40:04,2017-07-31 13:38:10,3131,银华中国梦30股票,股票型,2015-04-29,2017-07-30 10:40:04,2017-07-31 13:38:10


30324911


...

,基金代码,基金名称,相关链接,基金类型,规模（亿元）,任职时间,任职天数,任职回报,start_date,end_date,...,funds_type_x,funds_start_date_x,update_time_x,historical_price_time_x,funds_id_y,funds_name_full_y,funds_type_y,funds_start_date_y,update_time_y,historical_price_time_y
0,70031,嘉实全球房地产,估值图基金吧档案,QDII,0.51,2012-07-24 ~ 至今,5年又8天,35.24%,2012-07-24,9999-9-9,...,QDII,2012-07-24,2017-07-31 15:09:03,2017-07-31 16:33:16,4838,嘉实全球房地产,QDII,2012-07-24,2017-07-31 15:09:03,2017-07-31 16:33:16


30200977


,基金代码,基金名称,相关链接,基金类型,规模（亿元）,任职时间,任职天数,任职回报,start_date,end_date,...,funds_type_x,funds_start_date_x,update_time_x,historical_price_time_x,funds_id_y,funds_name_full_y,funds_type_y,funds_start_date_y,update_time_y,historical_price_time_y
0,660004,农银策略价值混合,估值图基金吧档案,混合型,3.77,2015-06-06 ~ 至今,2年又56天,-20.16%,2015-06-06,9999-9-9,...,混合型,2009-09-29,2017-07-31 02:37:03,2017-07-31 15:17:03,4088,农银策略价值混合,混合型,2009-09-29,2017-07-31 02:37:03,2017-07-31 15:17:03
1,462,农银主题轮动混合,估值图基金吧档案,混合型,5.86,2015-04-10 ~ 2016-12-28,1年又263天,-2.71%,2015-04-10,2016-12-28,...,混合型,2015-04-10,2017-07-31 08:05:03,2017-07-31 15:49:13,4416,农银主题轮动混合,混合型,2015-04-10,2017-07-31 08:05:03,2017-07-31 15:49:13
2,660003,农银平衡双利,估值图基金吧档案,混合型,3.54,2014-04-15 ~ 至今,3年又108天,70.37%,2014-04-15,9999-9-9,...,混合型,2009-04-08,2017-07-31 00:17:04,2017-07-31 15:03:03,3948,农银平衡双利,混合型,2009-04-08,2017-07-31 00:17:04,2017-07-31 15:03:03
3,259,农银区间收益混合,估值图基金吧档案,混合型,1.91,2013-08-21 ~ 至今,3年又345天,113.70%,2013-08-21,9999-9-9,...,混合型,2013-08-21,2017-07-30 22:32:03,2017-07-31 14:51:10,3843,农银区间收益混合,混合型,2013-08-21,2017-07-30 22:32:03,2017-07-31 14:51:10


30133951


IndexError: list index out of range